In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
#Category 
cate=[]
[cate.append(x[0][35:]) for x in os.walk('../input/plantdisease/PlantVillage/',topdown=True)]
cate=cate[1:]
cate

In [ ]:
#Image name .csv
names=[[],[]]
ind=0
for i in cate:
    for _,s,x in os.walk('../input/plantdisease/PlantVillage/'+i+'/',topdown=True):
        for f in x:
            if f[-4:]=='.jpg' or f[-4:]=='.JPG' or f[-5:]=='.jpeg':
                #There is 1 more pic with .jpqeg however , that pic have [4,256,256] dimension with diff from other [3,256,256] -> delete
                #../input/planvillageonefolder/Images/42f083e2-272d-4f83-ad9a-573ee90e50ec___Screen Shot 2015-05-06 at 4.01.13 PM.png
                names[0].append(f)
                names[1].append(ind)
            else:
                print(f)
    ind+=1
df=pd.DataFrame(names)
df=df.T
                #There is 1 more pic with .jpqeg however , that pic have [4,256,256] dimension with diff from other [3,256,256] -> delete
                #../input/planvillageonefolder/Images/42f083e2-272d-4f83-ad9a-573ee90e50ec___Screen Shot 2015-05-06 at 4.01.13 PM.png
df.head(20)

In [ ]:
df[0].unique()

In [ ]:
df.to_csv('annotation.csv')

In [ ]:
import matplotlib.pyplot as plt
plt.hist(df[1],bins=[x for x in range(0,len(cate))])
#-> distribution shift

In [ ]:
df.shape

In [ ]:
#Shuffle data
df=df.sample(frac=1)
#Split data train-test
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=1)
test,valid=train_test_split(test,test_size=0.5, random_state=1)

In [ ]:
train

In [ ]:
train.to_csv('train.csv')
train.shape


In [ ]:
test.to_csv('test.csv')
test.shape

In [ ]:
#Collect all images to 1 folder

In [ ]:
len(train)

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset,DataLoader

class plantdisease(Dataset):
    def __init__(self, train_df='./train.csv',test_df='./test.csv'
                 , img_dir='../input/planvillageonefolder/Images'
                 , transform=None, 
                 target_transform=None,isTrain=True,resize=False,isTest=False):
        if isTest==True:
            self.img_labels=test_df
            self.img_labels=test_df
        elif isTest==False:
            self.img_labels=train_df
            self.img_labels=train_df
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.isTrain=isTrain;
        self.resize=resize;
    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):

            img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
            image = (read_image(img_path).float())

            label = self.img_labels.iloc[idx, 1]
            if self.transform:
                image = self.transform(image)
            if self.target_transform:
                label = self.target_transform(label)
            return image, label

In [ ]:
import torchvision.transforms as transforms
train_dataset = plantdisease(train_df=train,isTest=False,transform=transforms.Compose([transforms.Resize((299,299))]))
test_dataset=plantdisease(test_df=test,isTest=True,transform=transforms.Compose([transforms.Resize((299,299))]))
train_loader=DataLoader(train_dataset, batch_size=5, shuffle=True, num_workers=1, pin_memory=False)
test_loader=DataLoader(test_dataset, batch_size=5, shuffle=True, num_workers=1, pin_memory=False)

In [ ]:
#Check unique sizes of dataset
#size=[0]
#for x,y in test_dataset:
#    if x.shape != size[-1]:
#        print(x.shape)
#        size.append(x.shape)

In [ ]:
#for x,y in test_dataset:
 #   a=1+1

In [ ]:
#import torchvision.models as models
#inception = models.inception_v3()


In [ ]:
#for x,y in train_loader:
    #print(x.shape)
    #break

In [ ]:
class metric():
  def __init__(self):
    self.train_lossHis=[]
    self.train_accuracy=[]
    self.test_accuracy=[]
    #cur_index will go after check evaluate
    self.loss_index=0
    self.train_index=0
    self.test_index=0
  def evaluate_by_setOfData(self):
    "Return train_lossHis,train_accuracy,test_accuracy"
    train_lossHis=np.mean(self.train_lossHis[self.loss_index:])
    train_accuracy=np.mean(self.train_accuracy[self.train_index:])
    test_accuracy=np.mean(self.test_accuracy[self.test_index:])
    self.loss_index=len(self.train_lossHis)-1
    self.train_index=len(self.train_accuracy)-1
    self.test_index=len(self.test_accuracy)-1
    return train_lossHis,train_accuracy,test_accuracy
  def append_loss(self,train_lossHis):
    self.train_lossHis.append(train_lossHis.item())
    return True
  def calculate_accuracy(self,Y_pred,y,isTest):
    if isTest:
      self.test_accuracy.append(self.accuracy(Y_pred,y))
      return self.test_accuracy[-1]
    else:
      self.train_accuracy.append(self.accuracy(Y_pred,y))
      return self.train_accuracy[-1]
  def accuracy(self,y_pred,y):
    return accuracy(y_pred,y).item()

In [ ]:
train

In [ ]:
!pip install torchmetrics
!pip install -U numpy

import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
from torchmetrics.functional import accuracy
from torchvision import transforms
import torchvision
from torch.utils.data import Dataset,DataLoader
from albumentations.core.composition import Compose, OneOf

In [ ]:
import torch
import torch.nn.functional as F
import torchmetrics
from torchmetrics.functional import accuracy
import torchvision.models as models

class GoogleNetV3():
  def __init__(self):
    from torch import nn
    googlenet = models.inception_v3()
    self.model = googlenet
    self.metric=0
    
  def train(self,batch,lr,num_epocs,train_data,test_data):
    trainer=torch.optim.SGD(self.model.parameters(), lr=lr)
    self.model.to(0)
    self.model.train(mode=True)
    self.metric=metric()
    #SGD
    for epoc in range(num_epocs):
      for x,y in train_data:
        trainer.zero_grad()
        x, y = x.to(0), y.to(0)
        y_hat=self.model(x)
        if type(y_hat)!=torch.Tensor:
            y_hat=y_hat.logits
        loss=nn.CrossEntropyLoss()(y_hat,y)
        self.metric.append_loss(loss)
        loss.backward()
        trainer.step()
      #Metric
      trainer.zero_grad()
      self.model.eval()
      for x,y in test_data:
        x, y = x.to(0), y.to(0)
        y_hat=self.model(x)
        
        test_accuracy=self.metric.calculate_accuracy(Y_pred=y_hat,y=y,isTest=True)
        
      for x,y in train_data:
        x, y = x.to(0), y.to(0)
        y_hat=self.model(x)

        train_accuracy=self.metric.calculate_accuracy(Y_pred=y_hat,y=y,isTest=False)
      loss_batch,accTest_batch,accTrain_batch=self.metric.evaluate_by_setOfData()
      print(f'epoch {epoc + 1},loss {float(loss_batch):f}, train Accuracy {float(accTest_batch):f},test Accuracy {float(accTrain_batch):f}')
      
      
  def accuracy(self,y_hat, y):
    return accuracy(y_hat,y)
model=GoogleNetV3()
model.train(batch=4,lr=0.001,num_epocs=5,train_data=train_loader,test_data=test_loader)

In [ ]:
#Validation 
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset,DataLoader


valid_data=plantdisease(train_df=valid,isTest=False,transform=transforms.Compose([transforms.Resize((299,299))]))
valid_loader=DataLoader(valid_data, batch_size=5, shuffle=True, num_workers=1, pin_memory=False)
model.model.eval()
eval_metric=metric()
for x,y in valid_loader:
    x, y = x.to(0), y.to(0)
    y_hat=model.model(x)
    test_accuracy=eval_metric.calculate_accuracy(Y_pred=y_hat,y=y,isTest=True)
    print(test_accuracy)